# Extract the keywords from the  text of the publications

In [1]:
import pandas as pd
import nltk

In [2]:
data_path = '/home/benjamin/Documents/memetracker/Data/csv/'
pickle_data_path = '/home/benjamin/Documents/memetracker/Data/pickle/'
series_name = 'marseille'
csvfile = data_path+series_name+'.csv'
Dataf = pd.read_csv(csvfile,sep=';')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (31,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# chargement des stopwords français
french_stopwords = set(stopwords.words('french'))
# add custom stop words
french_stopwords.update(['les','cette','http','https','fait','tout','tous','est'])

In [5]:
# Extract a subset of the data
filtered_data = Dataf[["date","text","title", "platform","hashtags"]].copy()
# the entries of the date column are dates:
filtered_data['date'] = pd.to_datetime(filtered_data['date'])#,infer_datetime_format=True)

In [6]:
# Twitter filter
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
tokens_tw = tknzr.tokenize(filtered_data.text[1])
print(filtered_data.text[1])
#print(tokens_tw)
#[word for word in tokens_tw if not 'http' in word]
[token for token in tokens_tw if ((token.lower() not in french_stopwords) 
                                                              and (len(token)>2)
                                                             and not 'http' in token)]

Marseille – nova série da Netflix http://t.co/lLG7hdg3DB #SdTV


['Marseille', 'nova', 'série', 'Netflix', '#SdTV']

In [7]:
def filter_text(dataframe):
    # filtering the texts
    texts = filtered_data.text
    filtered_text_list=texts.copy()
    for idx,row in dataframe.iterrows():
        #print(ind)
        texte = row.text
        platform = row.platform
        if (not pd.isnull(texte)):
            if platform == 'Twitter':
                tokens_tw = tknzr.tokenize(texte)
                #texte_tw = " ".join( tokens_tw )
                #texte_tw = re.findall(r"\w+", texte_tw)
                #texte_tw = " ".join( texte_tw )
                #tokens_fr = nltk.tokenize.word_tokenize(texte_tw, language='french')
                tokens_ns = [token for token in tokens_tw if ((token.lower() not in french_stopwords) 
                                                              and (len(token)>2)
                                                             and ('http' not in token)
                                                             and ('\'' not in token))]
                filtered_text = " ".join( tokens_ns )
            else:
                # filter out the chars that are not alphabet letters
                texte = re.findall(r"\w+", texte)
                texte = " ".join( texte )
                # Cut the text in tokens
                tokens_fr = nltk.tokenize.word_tokenize(texte, language='french')
                # filter out the french stop words
                # and
                # filter out the tokens smaller than 3 letters
                tokens_ns = [token for token in tokens_fr if ((token.lower() not in french_stopwords) 
                                                              and (len(token)>2))]
                filtered_text = " ".join( tokens_ns )
        else:
            filtered_text = " "
        filtered_text_list[idx] = filtered_text 
    return filtered_text_list

In [8]:
filtered_texts = filter_text(filtered_data)
filtered_data.loc[:,'filtered_text'] = filtered_texts

In [9]:
filtered_data.to_pickle(pickle_data_path+series_name+'_texts'+'.pkl')

In [10]:
filtered_data.filtered_text[2:25]

2     @ChristineBeaume @jl_burger fait penser serie ...
3     Originally Posted Durack vraiment bien aimé sa...
4     Originally Posted Sharn émerge oui bon hein sé...
5     émerge oui bon hein série Marseille 2015 Netfl...
6     Série #Netflix Marseille annonce casting févri...
7     015 marquera nouvelle ère Plusieurs historiens...
8     Sollicités Provence ils évoquent politique soc...
9     Ils mettent table Paul Wermus TILLINAC Manuel ...
10    janvier 2015 rigole bien mythique prison marse...
11    pourquoi pas5 janvier 2015 rejoins complètemen...
12    Vite dit grrr rigole bien mythique prison mars...
13    pourquoi pas5 janvier 2015 rejoins complètemen...
14    janvier 2015 lagazettedeputeaux Vincent Coquaz...
15    dentiste retrouvé égorgé cabinet Marseille Réd...
16    Prisons silence univers carcéral semble faire ...
17    financement films français jamais aussi diffic...
18    Boohoo avis Après Marseille embarque série art...
19    regarde série policière Marseille sais plu

## Extracting most frequent words with sci-kit learn

In [11]:
# loading the data as a list to be processed by sci-kit learn programs
clean_train_reviews = []
for text in filtered_data.filtered_text:
    clean_train_reviews.append(text)

In [12]:
import sklearn
import numpy as np

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [14]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print('Nb of documents: '+str(train_data_features.shape[0])+', '+
      'Nb of features: '+str(train_data_features.shape[1]))

Creating the bag of words...

Nb of documents: 32394, Nb of features: 5000


In [15]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

vocabdf = pd.DataFrame()
vocabdf['words'] = vocab
vocabdf['count'] = dist
vocabdf = vocabdf.sort_values('count',ascending=False)
vocabdf = vocabdf.reset_index(drop=True)
# Saving the data
vocabdf.to_pickle(pickle_data_path+series_name+'_vocab_bow'+'.pkl')
vocabdf.loc[0:50,:]

,words,count
0,marseille,39257
1,série,24919
2,netflix,20464
3,plus,12718
4,2016,10807
5,marseillenetflix,10587
6,depardieu,8659
7,france,8531
8,bien,7588
9,mai,7462


In [16]:
print("Creating the TFIDF vectors...\n")
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features_tfidf = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features_tfidf = train_data_features_tfidf.toarray()
vocab_tfidf = vectorizer.get_feature_names()
print('Nb of documents: '+str(train_data_features_tfidf.shape[0])+', '+
      'Nb of features: '+str(train_data_features_tfidf.shape[1]))

Creating the TFIDF vectors...

Nb of documents: 32394, Nb of features: 5000


In [17]:
# Sum up the counts of each vocabulary word
dist_tfidf = np.sum(train_data_features_tfidf, axis=0)

vocabdf = pd.DataFrame()
vocabdf['words'] = vocab_tfidf
vocabdf['count'] = dist_tfidf
vocabdf = vocabdf.sort_values('count',ascending=False)
vocabdf = vocabdf.reset_index(drop=True)
# Saving the data
vocabdf.to_pickle(pickle_data_path+series_name+'_vocab_tfidf'+'.pkl')
vocabdf.loc[0:50,:]

,words,count
0,marseille,2851.256342
1,marseillenetflix,2115.469048
2,série,2110.440963
3,netflix,1853.886850
4,serie,1734.384797
5,bien,1407.051365
6,ines_assia,1377.566286
7,truc,1376.855708
8,seul,1331.194686
9,marseilleserie,1205.915745


In [20]:
french_stopwords

{'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'as',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avons',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'c',
 'ce',
 'ces',
 'cette',
 'd',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'furent',
 'fus',
 'fusse',
 'fussent',
 'fusses',
 'fussiez',
 'fussions',
 'fut',
 'fûmes',
 'fût',
 'fûtes',
 'http',
 'https',
 'il',
 'j',
 'je',
 'l',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'm',
 'ma',
 'mais',
 'me',
 'mes',
 'moi',
 'mon',
 'même',
 'n',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ont',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 's',
 'sa',
 'se',
 'sera',
 'serai',
 'seraient',
 